# このJupyter Notebookの使い方
これはレシピ集として作成されています。最初に、初期設定セクションにて利用するElasticsearch環境やOpenAI環境、HuggingFace環境の接続情報を設定します。その後は、実行したいセクションA. B. C. ..から始めて順番にコマンドを実行してください。
そのためにセクション間で重複したコードが繰り返しあります。

# 1.初期設定

## ライブラリの有効化

In [1]:
%pip install -q elasticsearch langchain openai tiktoken


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pprint import pprint
import os
from getpass import getpass
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

## Elasticsearchの設定

In [3]:
ELASTIC_API_KEY = getpass("Elastic deployment API Key")
ELASTIC_CLOUD_ID = getpass("Elastic deployment Cloud ID")
ELASTIC_URL = getpass("Elastic deployment URL. No need if Cloud ID is provided.")
ELASTIC_USER = getpass("Elastic user. No need if API key is provided.")
ELASTIC_PASSWORD = getpass("Elastic password. No need if API key is provided.")

if ELASTIC_CLOUD_ID != '' and ELASTIC_API_KEY != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER == '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    # request_timeout=300,
    request_timeout=300
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")


pprint(es.info()) # should return cluster info

ObjectApiResponse({'name': 'instance-0000000023', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


## OpenAI APIの設定

In [4]:
num = getpass("Enter number: 1: OpenAI 2: Azure OpenAI 3: no LLM")
USE_OPENAI = num == "1"
USE_AZURE_OPENAI = num == "2"
os.environ["OPENAI_API_KEY"] = ""
if USE_OPENAI:
  os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")
  # os.environ["OPENAI_API_TYPE"] = "open_ai"
  # os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"

## Azure OpenAI APIの設定 (OpenAI APIを使う場合はスキップしてください)

In [5]:
if USE_AZURE_OPENAI:
    os.environ["OPENAI_API_BASE"] = getpass("Azure OpenAI API endpoint: ")
    os.environ["OPENAI_API_KEY"] = getpass("Azure OpenAI API key: ")
    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_VERSION"] = "2023-05-15"
    azureOpenAI_embedding_deployment = getpass("Name of Azure OpenAI deployment for embedding")
    azureOpenAI_chat_deployment = getpass("Name of Azure OpenAI deployment for chat")

## HuggingFace APIの設定

In [6]:
huggingface_api_Key = getpass("Enter your HF Inference API Key:\n\n")

## Elastic 上のMLモデルの設定

### 有効化するMLモデルの設定

In [7]:
enable_multilinguale5base = getpass("Enable ML model intfloat/multilingual-e5-base? Enter some key for 'Yes'.") != ""
enable_bertbasejapanesev3 = getpass("Enable ML model cl-tohoku/bert-base-japanese-v3? Enter some key for 'Yes'.") != ""
enable_elser = getpass("Enable ML model ELSER? Enter some key for 'Yes'.") != ""

In [16]:
%pip install -q eland elasticsearch transformers sentence_transformers
import tempfile
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from eland.ml.pytorch.transformers import elasticsearch_model_id
import time


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [133]:
def load_model(es_connection, model_id, task_type):
  with tempfile.TemporaryDirectory() as tmp_dir:
    print(f"Loading HuggingFace transformer tokenizer and model [{model_id}] for task [{task_type}]" )

    tm = TransformerModel(model_id=model_id, task_type=task_type)
    model_path, config, vocab_path = tm.save(tmp_dir)

    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    model_exists = es_connection.options(ignore_status=404).ml.get_trained_models(model_id=ptm.model_id).meta.status == 200

    if model_exists:
      print("Model has already been imported")
    else:
      print("Importing model")
      ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)
      print(f"Model successfully imported with id '{ptm.model_id}'")

def start_model(es_connection, model_id=None, es_model_id=None):
    if model_id:
      es_model_id = elasticsearch_model_id(model_id)
    if is_model_started(es_connection, es_model_id=es_model_id):
      print(f"Model '{es_model_id}' is already started")
    else:
      print(f"Starting model deployment '{es_model_id}'")
      es_connection.options(request_timeout=300).ml.start_trained_model_deployment(
            model_id=es_model_id, timeout="300s", wait_for="started")
      print(f"Model successfully started with id '{es_model_id}'")
    
def stop_model(es_connection, model_id=None, es_model_id=None):
    if model_id:
      es_model_id = elasticsearch_model_id(model_id)
    es_connection.options(request_timeout=300).ml.stop_trained_model_deployment(model_id=es_model_id, force=True)
    print("Stopping model deployment")

def is_model_started(es_connection, model_id=None, es_model_id=None):
    if model_id:
      es_model_id = elasticsearch_model_id(model_id)
    r = es_connection.ml.get_trained_models_stats(model_id=es_model_id)
    if r['trained_model_stats'][0].get('deployment_stats') != None:
      state = r['trained_model_stats'][0]['deployment_stats']['state']
      if state == 'started':
        print("Check: Model is started")
        return True
      elif state == 'starting':
        print("Check: Model is starting")
        return False
      else:
        print(f"Check: Model is in {state} state")
        return False
    else:
      print(f"Check: Model does not have deployment stats")
      return False

### Elastic MLにE5モデルをアップロード

In [128]:
if enable_multilinguale5base:
    load_model(es, "intfloat/multilingual-e5-base", "text_embedding")

Loading HuggingFace transformer tokenizer and model [intfloat/multilingual-e5-base] for task [text_embedding]
Model has already been imported


### Elastic ML E5モデルの開始

In [27]:
if enable_multilinguale5base:
    start_model(es, model_id="intfloat/multilingual-e5-base")


Check: Model is started
Model '{es_model_id}' is already started


### Elastic MLにTohoku Bertモデルをアップロード

In [131]:
if enable_bertbasejapanesev3:
    %pip install -q fugashi ipadic unidic_lite
    load_model(es, "cl-tohoku/bert-base-japanese-v3", "text_embedding")


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Loading HuggingFace transformer tokenizer and model [cl-tohoku/bert-base-japanese-v3] for task [text_embedding]


Importing model


100%|██████████| 423/423 [01:12<00:00,  5.85 parts/s]


Model successfully imported with id 'cl-tohoku__bert-base-japanese-v3'


### Elastic ML Tohoku Bertモデルの開始

In [29]:
if enable_bertbasejapanesev3: 
    start_model(es, model_id="cl-tohoku/bert-base-japanese-v3")

### Elastic ELSERモデルの開始

In [32]:
if enable_elser:
    start_model(es, es_model_id=".elser_model_1")
    # ptm = PyTorchModel(es, ".elser_model_1")

    # if not wait_until_started(es, ".elser_model_1"):
    # print("Starting model deployment")
    # ptm.start()
    # print(f"Model successfully started with id '.elser_model_1'")
    # wait_until_started(es, ".elser_model_1")

Check: Model is started
Model '.elser_model_1' is already started


# 2.検索ドキュメントのセットアップ


##　検索ドキュメントの設定

In [33]:
urls = [

    # --- サーチ Eコマース 記事 ------

    #検索分析を使ってEコマースソリューションを強化する
    "https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions",

    #Elasticで、個々のお客様に応じたEコマース向け検索エクスペリエンスを構築
    "https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic",

    #Elasticエンタープライズサーチでeコマースの競争力を高める6つの方法
    "https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce",

    #Elasticを使用して検索エクスペリエンスをパーソナライズする方法
    "https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic",

    # --- サーチ 画像検索 記事 ------
    #画像の類似検索の5つの技術要素
    "https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search",

    #Elasticで画像の類似検索を実装する方法
    "https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic",

    #Elasticsearchでの画像の類似検索の概要
    "https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic",

    # --- サーチ その他 記事 ------
    #検索候補の絞り込みと検索クエリの修正を実装する方法
    "https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections",


    # --- Observability 記事 ------

    #Elastic Common SchemaとOpenTelemetry — ベンダーロックインを避けながら、優れたオブザーバビリティとセキュリティを実現
    "https://www.elastic.co/jp/blog/ecs-elastic-common-schema-otel-opentelemetry-faq",

    #AIOpsビギナーズガイド
    "https://www.elastic.co/jp/blog/what-is-aiops-observability",

    # --- Security 記事 ------

    #Elastic SecurityのCloud Workload Protectionによるクラウド保護
    "https://www.elastic.co/jp/blog/secure-your-cloud-with-cloud-workload-protection-in-elastic-security",

    #教師ありおよび教師なしの機械学習を組み合わせてDGAを検知
    "https://www.elastic.co/jp/blog/supervised-and-unsupervised-machine-learning-for-dga-detection"
]

## 検索対象のドキュメントをダウンロードして整形

In [34]:
%pip install -qU pytest-playwright beautifulsoup4
!playwright install


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
zsh:1: /Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/bin/playwright: bad interpreter: /Users/nobuhikosekiya/lab/projects/es-langchain/.venv/bin/python: no such file or directory


## 2-A. WebBaseLoaderでWebページのテキストを抽出したDocumentを作る
このセクションでは、指定のWebページに含まれる全テキストを単純抽出して、Documentとして格納します。
今回はこれは見るだけで、実際は違うやり方の下の2-Bの方のデータを使います。

In [35]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(urls)
docs_notused = loader.load()
pprint(docs_notused, indent=4, width=30)

[   Document(page_content='検索分析にElasticを使ってEコマースソリューションを強化する | Elastic BlogSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：AIを導入して検索エクスペリエンスを変革続きを読むRWE社：再生可能エネルギー取引事業を強化続きを読むComcast社：エンジニアリングのスピード

## 2-B. WebページのHTMLからテキストを整形する
ここでは、以下の方法でメインのコンテンツだけを抽出するようにしています。
1. メインのコンテンツが含まれるHTMLのmainタグを抽出します。
2. 不要なscriptタグを除去しています。
3. さらにmainの中の必要なタグ h1, h3, h4, p, ulなどに絞ってテキストを抽出します。

結果のWebページのテキストが含まれたDocumentは**docs**のリストに格納されます。

In [36]:
from langchain.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from pprint import pprint

def extract_main_content(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    main_html = soup.find("main")
    scripts = main_html.find_all('script')
    for s in scripts:
        s.extract()

    if main_html:
        return main_html
    return None

loader = AsyncHtmlLoader(urls) # Use to load raw html into page_content
html_docs = loader.load()

for html_doc in html_docs:
#     pprint(html_doc.metadata)
    # print("======= 抽出前 =======")
    # pprint(html_doc.page_content[:1000])
    # print("======= 抽出後 =======")
    extraced_html = extract_main_content(f"<html><body>{html_doc.page_content}</body></html>")
    html_doc.page_content = f"<html><body>{extraced_html}</body></html>"
    # pprint(html_doc.page_content[:1000])
    # print("==============")
pprint(html_docs)

Fetching pages:   0%|          | 0/12 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 12/12 [00:07<00:00,  1.61it/s]


[Document(page_content='<html><body><main id="main-content" role="main"><div class="jsx-1686534253 blog-post-detail"><div class="jsx-1686534253 container mt-6"><div class="jsx-1686534253 row justify-content-center" id="sideTable"><div class="jsx-1686534253 col-12 col-lg-8"><div class="jsx-1686534253 main-header-block"><div class="jsx-1686534253 section" id="検索分析を使ってeコマースソリューションを強化する"><div class="jsx-1955866259 title-wrapper"><a class="jsx-1955866259" id="検索分析を使ってeコマースソリューションを強化する"></a><h1 class="jsx-1955866259" tag="H1">検索分析を使ってEコマースソリューションを強化する</h1></div></div><div class="jsx-1686534253 author-block mt-6"><div class="jsx-1686534253 author-desc d-flex"><div class="jsx-1686534253 d-flex flex-wrap mb-2"><p class="jsx-1686534253 medium">著者</p><a class="author-name" href="/blog/author/-the-elastic-enterprise-search-team"><p class="jsx-1686534253 medium"> The Elastic Enterprise Search team</p></a></div><time datetime="1660089600000">2022年8月10日</time></div><div class="jsx-1686534253 toc-btn"

In [37]:
from langchain.document_transformers import BeautifulSoupTransformer
bs_transformer = BeautifulSoupTransformer()

for html_doc in html_docs:
    extracted_text = bs_transformer.extract_tags(html_content=html_doc.page_content, tags=["h1","h3","h4","p","ul"])
    # pprint(extracted_text)
    html_doc.page_content = extracted_text
# docs_transformed = bs_transformer.transform_documents(docs, tags_to_extract=["a"])
# docs_transformed

docs = html_docs
for doc in docs:
    pprint(doc.metadata)
    print(doc.page_content)

{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデー

## テストしたい質問

In [38]:
questions = [
    "画像検索の方法について知りたい",
    "検索速度を速くする方法について知りたい",
    "Webサイトに検索を設けるにはどうすればいいか"
]

## テスト結果をまとめるスクリプト

In [39]:
import json

# Initialize a list to store the results
result_list = []

# Define the file path for data persistence
file_path = 'results.json'

def add_result(query, search_logic, result):
    # Check if a result with the same 'query' and 'search_logic' exists
    for item in result_list:
        if item['query'] == query and item['search_logic'] == search_logic:
            # Update the existing result
            item['result'] = result
            break
    else:
        # If no existing result found, add a new one
        result_list.append({'query': query, 'search_logic': search_logic, 'result': result})
    
    # Save the results to the file
    save_results()

def save_results():
    with open(file_path, 'w') as file:
        json.dump(result_list, file, ensure_ascii=False, indent=2)


def load_results():
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            result_list.clear()
            result_list.extend(data)
    except FileNotFoundError:
        # If the file doesn't exist, start with an empty results list
        result_list.clear()

# Load existing results from the file (if any)
load_results()

# Results after adding the examples
print(result_list)


[{'query': '画像検索の方法について知りたい', 'search_logic': 'bm25', 'result': {'query': '画像検索の方法について知りたい', 'result': ['{"metadata": {"source": "https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce"}, "highlight": {"page_content.kuromoji": ["Elasticエンタープライズサーチでeコマースの競争力を高める6つの<em>方法</em> Elastic Cloudの無料トライアルに登録 お使いの<em>検索</em>アプリケーションは、皆さんが思う以上にパワフルです。", "今回ご紹介する機能を使えば、<em>検索</em>データを活用して優れた顧客エクスペリエンスを構築できます。", "「たとえば、皆さんが会議に加わって『ユーザーがクエリXの後に何を<em>検索</em>しているか<em>知り</em>たい』と言ったときに、ユーザーが<em>検索</em>するキーワードのシーケンスを作成できます。", "コンバージョンと、コンバージョンに至る前の<em>検索</em>で区切ることも可能です」 <em>検索</em>と発見は重要ですが、独立して運用されているわけではありません。", "バックログには膨大な数の商品があり、それを最初の<em>検索</em>結果に表示させたい場合もあるでしょう。"]}}', '{"metadata": {"source": "https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search"}, "highlight": {"page_content.kuromoji": ["<em>画像</em>の類似<em>検索</em>の5つの技術要素 AI時代の<em>検索</em>ツールキット 推論：ユーザーのクエリをベクトル化する  <em>検索</em>：類似<em>画像</em> Elastic Cloudの無料トライアルに登録 著者 Rado

# 3.様々なサーチを試そう

## 3-A. Elasticsearchのキーワード検索 (BM25)

### インデックス作成
Kuromojiをアナライザーとして設定したElasticsearchのインデックスを作成します。

In [40]:
INDEX_NAME="test_index_bm25"

In [41]:

if es.indices.exists(index=INDEX_NAME):
    # If it exists, delete the index
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist.")

es.indices.create(
  index=INDEX_NAME,
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 0
      }
  }
)


Index 'test_index_bm25' deleted successfully.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index_bm25'})

In [42]:
es.indices.close(index=INDEX_NAME)

add_settings = {
  "index": {
    "analysis": {
      "char_filter": {
        "normalize": {
          "mode": "compose",
          "name": "nfkc",
          "type": "icu_normalizer"
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_index_synonym": {
          "type": "synonym",
          "lenient": "false",
          "synonyms": []
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "tokenizer": {
        "ja_kuromoji_tokenizer": {
          "mode": "search",
          "discard_compound_token": "true",
          "type": "kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

# Define a synonym set
synonyms_set = [
    # {
    #   "id": "test-1",
    #   "synonyms": "foo, bar"
    # },
    # {
    #   "id": "test-2",
    #   "synonyms": "test => check"
    # }
  ]

# Define the identifier for the synonym set
synonym_id = "ja_search_synonym-set"

from elasticsearch.client import SynonymsClient
# Call the put_synonym function to add or update the synonym set
response = SynonymsClient(es).put_synonym(id=synonym_id, synonyms_set=synonyms_set)

# Define the new settings you want to apply
add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_search_synonym": {
          "type": "synonym_graph",
          "synonyms_set": "ja_search_synonym-set",
          "updateable": "true"
        }
      },
      "analyzer": {
        "ja_kuromoji_index_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_index_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        },
        "ja_kuromoji_search_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_search_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

es.indices.open(index=INDEX_NAME, request_timeout=60)


/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_48614/1678612302.py:64: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import SynonymsClient
/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_48614/1678612302.py:119: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.open(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

In [43]:
es.indices.close(index=INDEX_NAME, request_timeout=60)

# Define the new mapping
add_mapping = {
  "properties": {
    "page_content": {
      "type": "text",
      "fields": {
        "kuromoji": {
          "type": "text",
          "analyzer": "ja_kuromoji_index_analyzer",
          "search_analyzer": "ja_kuromoji_search_analyzer"
        }
      }
    },
    "metadata": {
      "type": "object",
      "properties": {
        "source": {
          "type": "keyword"
        }
      }
    }
  }
}

es.indices.put_mapping(index=INDEX_NAME, body=add_mapping)

es.indices.open(index=INDEX_NAME)

/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_48614/2817861892.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.close(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

### インジェスト

In [44]:
from elasticsearch import Elasticsearch, helpers

# 前の実行で残っているDocumentはクリアしてからインジェストします
if es.indices.exists(index=INDEX_NAME):
    es.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

docs_json = [doc.to_json()["kwargs"] for doc in docs]
# print(docs_json)

index_docs = []
for doc_json in docs_json:
    index_docs.append({
        "_index": INDEX_NAME,
        "_source": doc_json,
    })

helpers.bulk(es, index_docs)

response = es.search(index=INDEX_NAME, query={"match_all": {}}, fields=["metadata"])
for hit in response['hits']['hits']:
    metadata = hit['_source']['metadata']
    print(f"metadata: {metadata}")

### サーチ

In [45]:
def do_search(query_text):
    query = {
        "bool": {
            "must": [
                {
                    "match": {
                        "page_content.kuromoji": {
                            "query": query_text,
                            "analyzer": "ja_kuromoji_search_analyzer"
                        }
                    }
                }
            ]
        }
    }
    highlight = {
        "fields": {
            "page_content.kuromoji": {}
        }
    }
    fields = ["metadata"]
    response = es.search(index=INDEX_NAME, query=query, fields=fields, highlight=highlight)

    results = []
    # Iterate through the search results and access the fields you want
    for hit in response['hits']['hits']:
        metadata = hit['_source']['metadata']
        score = hit['_score']
        highlight = hit['highlight']
        print(f"_score: {score}, metadata: {metadata}")
        pprint(f"{highlight}")
        r = {
            "metadata": metadata,
            "highlight": highlight
        }
        results.append(json.dumps(r, ensure_ascii=False))

    return {"query": query_text, "result": results}

In [46]:
for query in questions:
    print(query)
    result = do_search(query)
    add_result(search_logic="bm25", query=query, result=result)

画像検索の方法について知りたい
検索速度を速くする方法について知りたい
Webサイトに検索を設けるにはどうすればいいか


## 3-B. （Open AI Embeddingを利用） Elasticsearchのセマンティック検索
LangchainのElasticsearchStoreというクラスを使って色々なAI/MLモデルを切り替えながらElasticsearchのベクトル検索をテストしていくことができます。

参考：https://python.langchain.com/docs/integrations/vectorstores/elasticsearch

### インジェスト
補足：ElasticsearchStoreの中で自動的にインデックスの作成も行われています。

In [47]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.openai import OpenAIEmbeddings

    if USE_AZURE_OPENAI:
        embedding = OpenAIEmbeddings(deployment=azureOpenAI_embedding_deployment)
    else:
        embedding = OpenAIEmbeddings(openai_api_type="open_ai")

    db_openai = ElasticsearchStore(
        es_connection=es,
        index_name="test_index_openai",
        embedding=embedding
    )

Skipping because model is not enabled


In [48]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    # 前の実行で残っているDocumentはクリアしてからインジェストします
    if es.indices.exists(index="test_index_openai"):
        db_openai.client.delete_by_query(index="test_index_openai", body={"query": {"match_all": {}}})
    db_openai.add_documents(docs)
    db_openai.client.indices.refresh(index="test_index_openai")

Skipping because model is not enabled


### サーチ

In [49]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_openai.similarity_search(query)
        [print(element.metadata) for element in results]
        add_result(search_logic="vector_openai_embeddings", query=query, result=[element.metadata for element in results])

Skipping because model is not enabled


## 3-C. (HuggingFaceにあるE5モデルを利用） Elasticsearchのセマンティック検索

#### インジェスト

In [50]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
        
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

    MODEL_ID="intfloat/multilingual-e5-base"

    if not huggingface_api_Key:
        raise Exception("huggingface_api_Key is not set")

    embedding = HuggingFaceInferenceAPIEmbeddings(
        api_key=huggingface_api_Key,
        model_name=MODEL_ID
    )
    db_huggingface_e5 = ElasticsearchStore(
        es_connection=es,
        index_name="test_index_huggingface_e5",
        embedding=embedding
    )


Skipping because model is not enabled


In [51]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    if db_huggingface_e5.client.indices.exists(index="test_index_huggingface_e5"):
        db_huggingface_e5.client.delete_by_query(index="test_index_huggingface_e5", body={"query": {"match_all": {}}})
    db_huggingface_e5.add_documents(docs)
    db_huggingface_e5.client.indices.refresh(index="test_index_huggingface_e5")

Skipping because model is not enabled


#### サーチ

In [52]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_huggingface_e5.similarity_search(query)
        [print(element.metadata) for element in results]
        add_result(search_logic="vector_huggingface_e5", query=query, result=[element.metadata for element in results])

Skipping because model is not enabled


## 3-D. (ElasticsearchにアップしたE5モデルを利用） Elasticsearchのセマンティック検索

### インジェスト

In [53]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    MODEL_ID="intfloat__multilingual-e5-base"

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=MODEL_ID
    )

    db_esml_e5 = ElasticsearchStore(
        es_connection=es,
        index_name="test_index_esml_e5",
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy()
    )

Check: Model is started


In [54]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    if db_esml_e5.client.indices.exists(index="test_index_esml_e5"):
        db_esml_e5.client.delete_by_query(index="test_index_esml_e5", body={"query": {"match_all": {}}})

    for doc in docs:
        db_esml_e5.add_documents([doc])
        db_esml_e5.client.indices.refresh(index="test_index_esml_e5")
        print('.', end='')

Check: Model is started


............

### サーチ

In [55]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:

    for query in questions:
        print(query)
        results = db_esml_e5.similarity_search(query)
        [print(element.metadata) for element in results]
        add_result(search_logic="vector_elastic_e5", query=query, result=[element.metadata for element in results])

Check: Model is started
画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/us

## 3-E. Elasticsearch RRF ハイブリッド検索

### インジェスト

In [56]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    MODEL_ID="intfloat__multilingual-e5-base"

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=MODEL_ID
    )

    db_esml_e5_hybrid = ElasticsearchStore(
        es_connection=es.options(request_timeout=3600),
        index_name="test_index_esml_e5_hybrid",
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
    )

Check: Model is started


In [57]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:

    if db_esml_e5_hybrid.client.indices.exists(index="test_index_esml_e5_hybrid"):
        db_esml_e5_hybrid.client.delete_by_query(index="test_index_esml_e5_hybrid", body={"query": {"match_all": {}}})
    print(len(docs))
    db_esml_e5_hybrid.add_documents(docs)
    db_esml_e5_hybrid.client.indices.refresh(index="test_index_esml_e5_hybrid")

Check: Model is started
12


### サーチ

In [58]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_esml_e5_hybrid.similarity_search(query)
        [print(element.metadata) for element in results]
        add_result(search_logic="hybrid_elastic_bm25_e5", query=query, result=[element.metadata for element in results])

Check: Model is started
画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.

## 3-F. (HuggingFaceにある英語用モデルを利用） Elasticsearchのセマンティック検索

### インジェスト

In [59]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

    MODEL_ID="sentence-transformers/all-distilroberta-v1"

    if not huggingface_api_Key:
        raise Exception("huggingface_api_Key is not set")

    embedding = HuggingFaceInferenceAPIEmbeddings(
        api_key=huggingface_api_Key,
        model_name=MODEL_ID
    )
    db_huggingface_distilroberta = ElasticsearchStore(
        es_connection=es,
        index_name="test_index_huggingface_distilroberta",
        embedding=embedding
    )


Skipping because model is not enabled


In [60]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    if db_huggingface_distilroberta.client.indices.exists(index="test_index_huggingface_distilroberta"):
        db_huggingface_distilroberta.client.delete_by_query(index="test_index_huggingface_distilroberta", body={"query": {"match_all": {}}})
    db_huggingface_distilroberta.add_documents(docs)
    db_huggingface_distilroberta.client.indices.refresh(index="test_index_huggingface_distilroberta")

Skipping because model is not enabled


### サーチ

In [61]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_huggingface_distilroberta.similarity_search(query)
        [print(element.metadata) for element in results]
        add_result(search_logic="vector_distilroberta", query=query, result=[element.metadata for element in results])

Skipping because model is not enabled


## 3-G.（Elastic ELSERを利用） ElasticsearchcでSparse Vector検索

### インジェスト

In [62]:
if not is_model_started(es, ".elser_model_1"):
    print("Skipping because model is not enabled")
else:

    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

    db_elser = ElasticsearchStore(
        es_connection=es,
        index_name="test_index_elser",
        strategy=ElasticsearchStore.SparseVectorRetrievalStrategy()
    )

Check: Model is started


In [63]:
if not is_model_started(es, ".elser_model_1"):
    print("Skipping because model is not enabled")
else:
    
    if es.indices.exists(index="test_index_elser"):
        db_elser.client.delete_by_query(index="test_index_elser", body={"query": {"match_all": {}}})
    db_elser.add_documents(docs)
    db_elser.client.indices.refresh(index="test_index_elser")

Check: Model is started


### サーチ

In [64]:
if not is_model_started(es, ".elser_model_1"):
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_elser.similarity_search(query)
        [print(element.metadata) for element in results]
        add_result(search_logic="sparsevector_elastic_elser", query=query, result=[element.metadata for element in results])

Check: Model is started
画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/supervised-and-unsupervised-machine-learning-for-dga-detection'}
{'so

## 3-I. (ElasticsearchにアップしたTohoku BERT Japanese v3モデルを利用) Elasticのセマンティック検索

In [65]:
if not is_model_started(es, "cl-tohoku__bert-base-japanese-v3"):
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    MODEL_ID="cl-tohoku__bert-base-japanese-v3"

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=MODEL_ID
    )

    db_esml_tohokubertv3 = ElasticsearchStore(
        es_connection=es,
        index_name="test_index_esml_tohokubertv3",
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy()
    )

Check: Model does not have deployment stats
Skipping because model is not enabled


### インジェスト

In [66]:
if not is_model_started(es, "cl-tohoku__bert-base-japanese-v3"):
    print("Skipping because model is not enabled")
else:
    
    if db_esml_tohokubertv3.client.indices.exists(index="test_index_esml_tohokubertv3"):
        db_esml_tohokubertv3.client.delete_by_query(index="test_index_esml_tohokubertv3", body={"query": {"match_all": {}}})
    for doc in docs:
        db_esml_tohokubertv3.add_documents([doc])
        db_esml_tohokubertv3.client.indices.refresh(index="test_index_esml_tohokubertv3")
        print(".", end='')

Check: Model does not have deployment stats
Skipping because model is not enabled


### サーチ

In [67]:
if not is_model_started(es, "cl-tohoku__bert-base-japanese-v3"):
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_esml_tohokubertv3.similarity_search(query)
        [print(element.metadata) for element in results]
        add_result(search_logic="esml_tohokubertv3", query=query, result=[element.metadata for element in results])

Check: Model does not have deployment stats
Skipping because model is not enabled


# 4. RAG検索: LLMを組み合わせてみよう

## 4-1. 文書をChunkにSplitする

### CharacterTextSplitterでSplitする

In [68]:
from langchain.text_splitter import CharacterTextSplitter

print("---- All DOCS ----")
pprint(docs)
text_splitter = CharacterTextSplitter(separator = "\s\s", chunk_size = 500, chunk_overlap = 200, is_separator_regex=True)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10 ----")
for split in all_splits[:10]:
    pprint(split.metadata['source'])
    pprint(split.page_content)

Created a chunk of size 532, which is longer than the specified 500
Created a chunk of size 632, which is longer than the specified 500
Created a chunk of size 558, which is longer than the specified 500
Created a chunk of size 2510, which is longer than the specified 500
Created a chunk of size 505, which is longer than the specified 500
Created a chunk of size 836, which is longer than the specified 500
Created a chunk of size 872, which is longer than the specified 500
Created a chunk of size 784, which is longer than the specified 500
Created a chunk of size 686, which is longer than the specified 500
Created a chunk of size 884, which is longer than the specified 500
Created a chunk of size 1101, which is longer than the specified 500
Created a chunk of size 722, which is longer than the specified 500
Created a chunk of size 1652, which is longer than the specified 500
Created a chunk of size 503, which is longer than the specified 500
Created a chunk of size 663, which is longer 

---- All DOCS ----
[Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデータは特に、ユーザーの地理情報その他のプロファイル情報など、各種メタデータと合わせて検討すると有用な情報に変わります。た

### RecursiveCharacterTextSplitterでSplitする

In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("---- ALL DOCS----")
pprint(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10----")

for split in all_splits[:10]:
    pprint(split.metadata['source'])
    pprint(split.page_content)

---- ALL DOCS----
[Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデータは特に、ユーザーの地理情報その他のプロファイル情報など、各種メタデータと合わせて検討すると有用な情報に変わります。たと

## 4-２. RetrievalQA chainでQA回答をする

### RetrievalQA chainのファンクションを作成

In [70]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI,AzureChatOpenAI

def ask_llm(vectorstore, question, template=None):
    if not template:
        template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible.
    Always say "thanks for asking!" at the end of the answer.
    {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    if USE_AZURE_OPENAI:
        llm = AzureChatOpenAI(deployment_name=azureOpenAI_chat_deployment)
    elif USE_OPENAI:
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    qa_chain = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
    )
    result = qa_chain({"query": question})
    pprint("------------ result -----------")
    pprint(result["result"])
    pprint("------------ source_documents -----------")
    pprint(result['source_documents'])
    return {"query": question, "result": f"{result['result']}\n{result['source_documents']}"}

## 4-３. Retrieverを作成して、QAする

### OpenAI Embedding

In [71]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.openai import OpenAIEmbeddings

    if USE_AZURE_OPENAI:
        embedding = OpenAIEmbeddings(deployment=azureOpenAI_embedding_deployment)
    else:
        embedding = OpenAIEmbeddings(openai_api_type="open_ai")

    pprint(es.info())
    db_split_openai = ElasticsearchStore(
        es_connection=es,
        index_name="test_split_index_openai",
        embedding=embedding
    )

Skipping because model is not enabled


#### インジェスト

In [72]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    if es.indices.exists(index="test_split_index_openai"):
        db_split_openai.client.delete_by_query(index="test_split_index_openai", body={"query": {"match_all": {}}})
    db_split_openai.add_documents(all_splits)
    db_split_openai.client.indices.refresh(index="test_split_index_openai")

Skipping because model is not enabled


#### サーチ

In [73]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_split_openai.similarity_search(query, k=5)
        for element in results:
            print(element.metadata) 
            print(element.page_content) 

Skipping because model is not enabled


#### RAGサーチ

In [74]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        result = ask_llm(vectorstore=db_split_openai, question=query)
        add_result(search_logic="rag_openai_embedding", query=query, result=result)


Skipping because model is not enabled


### HuggingFace API: E5モデル

In [75]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

    if not huggingface_api_Key:
        raise Exception("huggingface_api_Key is not set")

    MODEL_ID="intfloat/multilingual-e5-base"

    embedding = HuggingFaceInferenceAPIEmbeddings(
        api_key=huggingface_api_Key,
        model_name=MODEL_ID
    )
    db_split_huggingface_e5 = ElasticsearchStore(
        es_connection=es,
        index_name="test_split_index_huggingface_e5",
        embedding=embedding
    )

Skipping because model is not enabled


#### インジェスト

In [76]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    if es.indices.exists(index="test_split_index_huggingface_e5"):
        db_split_huggingface_e5.client.delete_by_query(index="test_split_index_huggingface_e5", body={"query": {"match_all": {}}})
    db_split_huggingface_e5.add_documents(all_splits)
    db_split_huggingface_e5.client.indices.refresh(index="test_split_index_huggingface_e5")

Skipping because model is not enabled


#### サーチ

In [77]:
if huggingface_api_Key == "":
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        results = db_split_huggingface_e5.similarity_search(query, k=5)
        for element in results:
            print(element.metadata) 
            print(element.page_content) 

Skipping because model is not enabled


#### RAGサーチ

In [78]:
if huggingface_api_Key == "" or os.environ['OPENAI_API_KEY'] == "":
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        result = ask_llm(vectorstore=db_split_huggingface_e5, question=query)
        add_result(search_logic="rag_huggingface_e5", query=query, result=result)

Skipping because model is not enabled


### Elastic MLのE5モデル

In [79]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:

    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    MODEL_ID="intfloat__multilingual-e5-base"

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=MODEL_ID,
        input_field="text_field"
    )

    db_split_esml_e5 = ElasticsearchStore(
        es_connection=es,
        index_name="test_split_index_esml_e5",
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy()
    )

Check: Model is started


#### インジェスト

In [80]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    if db_split_esml_e5.client.indices.exists(index="test_split_index_esml_e5"):
        db_split_esml_e5.client.delete_by_query(index="test_split_index_esml_e5", body={"query": {"match_all": {}}})

    for split in all_splits:
        db_split_esml_e5.add_documents([split])
        print(".", end='')
        db_split_esml_e5.client.indices.refresh(index="test_split_index_esml_e5")

Check: Model is started


.................................................................................................................................................................

#### RAGサーチ

In [81]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ['OPENAI_API_KEY'] == '':
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        result = ask_llm(vectorstore=db_split_esml_e5, question=query)
        add_result(search_logic="rag_elasticml_e5", query=query, result=result)

Check: Model is started
Skipping because model is not enabled


### Elastic RRFハイブリッド検索

In [82]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    MODEL_ID="intfloat__multilingual-e5-base"

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=MODEL_ID,
        input_field="text_field"
    )

    db_split_esml_e5_hybrid = ElasticsearchStore(
        es_connection=es,
        index_name="test_split_index_esml_e5_hybrid",
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
    )

Check: Model is started


#### インジェスト

In [83]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    if db_split_esml_e5_hybrid.client.indices.exists(index="test_split_index_esml_e5_hybrid"):
        db_split_esml_e5_hybrid.client.delete_by_query(index="test_split_index_esml_e5_hybrid", body={"query": {"match_all": {}}})

    for split in all_splits:
        db_split_esml_e5_hybrid.add_documents([split])
        print(".", end='')
        db_split_esml_e5_hybrid.client.indices.refresh(index="test_split_index_esml_e5_hybrid")

Check: Model is started


.................................................................................................................................................................

#### RAGサーチ

In [84]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ['OPENAI_API_KEY'] == '':
    print("Skipping because model is not enabled")
else:
    
    for query in questions:
        print(query)
        result = ask_llm(vectorstore=db_split_esml_e5_hybrid, question=query)
        add_result(search_logic="rag_elasticml_hybrid_e5", query=query, result=result)

Check: Model is started
Skipping because model is not enabled


### Elastic ELSER

In [85]:
if not is_model_started(es, ".elser_model_1"):
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore

    db_split_elser = ElasticsearchStore(
        es_connection=es,
        index_name="test_split_index_elser",
        strategy=ElasticsearchStore.SparseVectorRetrievalStrategy()
    )

Check: Model is started


#### インジェスト

In [86]:
if not is_model_started(es, ".elser_model_1"):
    print("Skipping because model is not enabled")
else:
    
    if db_split_elser.client.indices.exists(index="test_split_index_elser"):
        db_split_elser.client.delete_by_query(index="test_split_index_elser", body={"query": {"match_all": {}}})

    for split in all_splits:
        db_split_elser.add_documents([split])
        print(".", end='')
        db_split_elser.client.indices.refresh(index="test_split_index_elser")

Check: Model is started


.................................................................................................................................................................

In [87]:
if not is_model_started(es, ".elser_model_1"):
    print("Skipping because model is not enabled")
else:
        
    %pip install -q numpy
    import numpy as np
    # Define the size of sublists
    chunk_size = 10

    # Using numpy.array_split to split the list
    split_list = np.array_split(all_splits, len(all_splits) // chunk_size)

    if es.indices.exists(index="test_split_index_elser"):
        db_split_elser.client.delete_by_query(index="test_split_index_elser", body={"query": {"match_all": {}}})

    for splits in split_list:
        db_split_elser.add_documents(splits)
        db_split_elser.client.indices.refresh(index="test_split_index_elser")

Check: Model is started

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### RAGサーチ

In [88]:
if not is_model_started(es, ".elser_model_1") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
        
    for query in questions:
        print(query)
        result = ask_llm(vectorstore=db_split_elser, question=query)
        add_result(search_logic="rag_esml_elser", query=query, result=result)

Check: Model is started
Skipping because model is not enabled


# 5. より高度なRAGパターン

## 5-1. Self Query Retrieverで、質問文から検索のフィルターをかける

In [89]:
!python3 -m pip install -qU lark elasticsearch langchain openai

from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.llms import OpenAI, AzureOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### 文書にタグづけする

In [90]:
metadatas = [
    #検索分析を使ってEコマースソリューションを強化する
    { "url": "https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions", "tags": ["Eコマース","概要"] },

    #Elasticで、個々のお客様に応じたEコマース向け検索エクスペリエンスを構築
    { "url": "https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic", "tags": ["Eコマース", "概要"] },

    #Elastic App SearchでSpring Bootを使用する
    { "url": "https://www.elastic.co/jp/blog/using-spring-boot-with-elastic-app-search", "tags": ["実装方法"] },

    #Elasticエンタープライズサーチでeコマースの競争力を高める6つの方法
    { "url": "https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce", "tags": ["Eコマース","概要"] },

    #Elasticを使用して検索エクスペリエンスをパーソナライズする方法
    { "url": "https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic", "tags": ["Eコマース","概要"] },

    #検索候補の絞り込みと検索クエリの修正を実装する方法
    { "url": "https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections", "tags": ["実装方法"] },

    #画像の類似検索の5つの技術要素
    { "url": "https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search", "tags": ["画像検索","概要"] },

    #Elasticで画像の類似検索を実装する方法
    { "url": "https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic", "tags": ["画像検索","実装方法"] },

    #Elasticsearchでの画像の類似検索の概要
    { "url": "https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic", "tags": ["画像検索","概要"] },
]

In [91]:
for doc in docs:
    for metadata in metadatas:
        if doc.metadata['source'] == metadata['url']:
            doc.metadata['tags'] = metadata['tags']
pprint(docs[0])

Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデータは特に、ユーザーの地理情報その他のプロファイル情報など、各種メタデータと合わせて検討すると有用な情報に変わります。たとえば、運営しているEコマースサイトの上

### インジェスト

In [92]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    MODEL_ID="intfloat__multilingual-e5-base"

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=MODEL_ID
    )

    db_esml_e5_hybrid_tags = ElasticsearchStore(
        es_connection=es,
        index_name="test_index_esml_e5_hybrid_tags",
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
    )


Check: Model is started


In [93]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    if db_esml_e5_hybrid_tags.client.indices.exists(index="test_index_esml_e5_hybrid_tags"):
        db_esml_e5_hybrid_tags.client.delete_by_query(index="test_index_esml_e5_hybrid_tags", body={"query": {"match_all": {}}})

    for doc in docs:
        db_esml_e5_hybrid_tags.add_documents([doc])
        db_esml_e5_hybrid_tags.client.indices.refresh(index="test_index_esml_e5_hybrid_tags")
        print(".", end="")

Check: Model is started


............

### サーチ(手動でフィルターかける）

In [94]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    query="画像検索について知りたい"
    results = db_esml_e5_hybrid_tags.similarity_search(query, filter=[{ 'match': { "metadata.tags": "実装方法" }}], k=10)
    [print(element.metadata) for element in results]

Check: Model is started
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic', 'tags': ['画像検索', '実装方法']}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections', 'tags': ['実装方法']}


### サーチ (NLPでフィルターかける)

In [95]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    # Add details about metadata fields
    metadata_field_info = [
        AttributeInfo(
            name="tags",
            description="文書の特徴を表すタグ",
            type="string or list[string]",
        )
    ]

    document_content_description = "Elasticsearchの検索に関するブログ。入門編か実装編かののタグが付与されている。特定のインダストリーに関するものはそのインダストリーのタグが付与されている。特定のソリューションに関するものはそのタグが付与されている。"

    # Set up openAI llm with sampling temperature 0
    if USE_AZURE_OPENAI:
        llm = AzureOpenAI(deployment_name="text-davinci-003")
    else:
        llm = OpenAI(temperature=0)

    # instantiate retriever
    selfquery_retriever = SelfQueryRetriever.from_llm(
        llm, db_esml_e5_hybrid_tags, document_content_description, metadata_field_info, verbose=True
    )

Check: Model is started
Skipping because model is not enabled


In [96]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.self_query").setLevel(logging.INFO)

In [97]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ['OPENAI_API_KEY'] == "":
    print("Skipping because model is not enabled")
else:
    
    results = selfquery_retriever.get_relevant_documents("画像検索について知りたい")
    [print(element.metadata) for element in results]

Check: Model is started
Skipping because model is not enabled


In [98]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ['OPENAI_API_KEY'] == "":
    print("Skipping because model is not enabled")
else:
    
    results = selfquery_retriever.get_relevant_documents("画像検索の詳しい実装方法について知りたい")
    [print(element.metadata) for element in results]

Check: Model is started
Skipping because model is not enabled


In [99]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ['OPENAI_API_KEY'] == "":
    print("Skipping because model is not enabled")
else:
    
    results = selfquery_retriever.get_relevant_documents("Eコマースにおける画像検索について知りたい")
    [print(element.metadata) for element in results]

Check: Model is started
Skipping because model is not enabled


In [100]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ['OPENAI_API_KEY'] == "":
    print("Skipping because model is not enabled")
else:
    
    results = selfquery_retriever.get_relevant_documents("Eコマースにおける画像検索の詳しい実装方法について知りたい")
    [print(element.metadata) for element in results]

Check: Model is started
Skipping because model is not enabled


### RAG用のSplitドキュメントにもタグづけする

In [101]:
for split in all_splits:
    for metadata in metadatas:
        if split.metadata['source'] == metadata['url']:
            split.metadata['tags'] = metadata['tags']
pprint(all_splits[0])
pprint(all_splits[1])

Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。', metadata={'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions', 'tags': ['Eコマース', '概要']})
Document(page_content='お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。', metadata={'sou

### RAG用のSplitデータをインジェスト

In [102]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
        
    from langchain.vectorstores.elasticsearch import ElasticsearchStore
    from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

    MODEL_ID="intfloat__multilingual-e5-base"

    embedding = ElasticsearchEmbeddings.from_es_connection(
        es_connection=es,
        model_id=MODEL_ID
    )

    db_split_esml_e5_hybrid_tags = ElasticsearchStore(
        es_connection=es,
        index_name="test_split_index_esml_e5_hybrid_tags",
        embedding=embedding,
        strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
    )

Check: Model is started


In [103]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    if db_split_esml_e5_hybrid_tags.client.indices.exists(index="test_split_index_esml_e5_hybrid_tags"):
        db_split_esml_e5_hybrid_tags.client.delete_by_query(index="test_split_index_esml_e5_hybrid_tags", body={"query": {"match_all": {}}})

    for split in all_splits:
        db_split_esml_e5_hybrid_tags.add_documents([split])
        print(".", end='')
        db_split_esml_e5_hybrid_tags.client.indices.refresh(index="test_split_index_esml_e5_hybrid_tags")

Check: Model is started


.................................................................................................................................................................

### RAGでのサーチ

In [104]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    # Add details about metadata fields
    metadata_field_info = [
        AttributeInfo(
            name="tags",
            description="文書の特徴を表すタグ",
            type="string or list[string]",
        )
    ]

    document_content_description = "Elasticsearchの検索に関するブログ。入門編か実装編かののタグが付与されている。特定のインダストリーに関するものはそのインダストリーのタグが付与されている。特定のソリューションに関するものはそのタグが付与されている。"

    # Set up openAI llm with sampling temperature 0
    if USE_AZURE_OPENAI:
        llm = AzureOpenAI(deployment_name="text-davinci-003")
    else:
        llm = OpenAI(temperature=0)

    # instantiate retriever
    selfquery_retriever = SelfQueryRetriever.from_llm(
        llm, db_split_esml_e5_hybrid_tags, document_content_description, metadata_field_info, verbose=True
    )

Check: Model is started
Skipping because model is not enabled


In [105]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

def ask_llm_with_selfquery(selfquery_retriever, question, template=None):
    if not template:
        template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible.
    Always say "thanks for asking!" at the end of the answer.
    {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    if USE_AZURE_OPENAI:
        llm = AzureChatOpenAI(deployment_name=azureOpenAI_chat_deployment)
    else:
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    qa_chain = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff",
        retriever=selfquery_retriever,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
    )
    result = qa_chain({"query": question})
    pprint("------------ result -----------")
    pprint(result["result"])
    pprint("------------ source_documents -----------")
    pprint(result['source_documents'])
    return {"query": question, "result": f"{result['result']}\n{result['source_documents']}"}

In [106]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"] == "":
    print("Skipping because model is not enabled")
else:
    
    query="画像検索の詳しい実装方法について知りたい"
    print(query)
    result = ask_llm_with_selfquery(selfquery_retriever=selfquery_retriever, question=query)
    add_result(search_logic="rag_esml_e5_hybrid_selfquery", query=query, result=result)


Check: Model is started
Skipping because model is not enabled


## 5-2. Agentsを使い、外部プログラムとしてElasticsearchのデータを検索させる

In [107]:
# Import things that are needed generically
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [108]:
from langchain.tools import tool

@tool
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return f"Results for query {query}"

In [109]:
from langchain.tools import tool


@tool
def elasticsearch_doc_counts_api(query: str) -> str:
    """Searches the API for the query."""
    query = {
        "bool": {
            "must": [
                {
                    "match": {
                        "page_content.kuromoji": {
                            "query": query,
                            "analyzer": "ja_kuromoji_search_analyzer"
                        }
                    }
                }
            ]
        }
    }
    fields = ["metadata", "page_content"]
    response = es.search(index=INDEX_NAME, query=query, fields=fields, size=10)

    # Iterate through the search results and access the fields you want
    for hit in response['hits']['hits']:
        metadata = hit['_source']['metadata']
        page_content = hit['_source']['page_content']
        score = hit['_score']
        print(f"_score: {score}, metadata: {metadata}")
    
    return len(response['hits']['hits'])


In [110]:
from langchain.tools import tool

def vector_doc_counts_api_impl(vectorstore, query):
    results = vectorstore.similarity_search(query, 50)
    source_doc_count = set() 
    for element in results:
        print(element.metadata['source'])
        source_doc_count.add(element.metadata['source'])

    return len(source_doc_count)

@tool
def vector_doc_counts_api(query: str) -> str:
    """Searches the API for the query."""
    return vector_doc_counts_api_impl(vectorstore=db_split_openai, query=query)

In [111]:
# Load the tool configs that are needed.
tools = [
    Tool.from_function(
        func=vector_doc_counts_api.run,
        name="Search with vector search",
        description="ベクトル検索でクエリーして確認できた検索対象の文書の数を返すAPI",
    ),
    Tool.from_function(
        func=elasticsearch_doc_counts_api.run,
        name="Elasticsearch",
        description="Elasticsearchでクエリーして確認できた検索対象の文書の数を返すAPI",
    )
    ]

In [112]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    if USE_AZURE_OPENAI:
        llm = AzureOpenAI(deployment_name="text-davinci-003")
    else:
        llm = OpenAI(temperature=0)
    # Construct the agent. We will use the default agent type here.
    # See documentation for a full list of options.
    agent = initialize_agent(
        tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )

Skipping because model is not enabled


In [113]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    agent.run(
        "Eコマースに関する文書は何件ありますか?"
    )

Skipping because model is not enabled


In [114]:
if os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    agent.run(
        "Eコマースに関する文書は何件ありますか？全ての検索方法での結果をそれぞれ教えてください"
    )

Skipping because model is not enabled


## 5-3. チャット化

In [115]:
if not is_model_started(es, "intfloat__multilingual-e5-base"):
    print("Skipping because model is not enabled")
else:
    
    vector_store = db_split_esml_e5_hybrid

Check: Model is started


In [116]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    
    from langchain.llms import OpenAI
    from langchain.chains import ConversationalRetrievalChain
    from langchain.memory import ElasticsearchChatMessageHistory
    from uuid import uuid4


    retriever = vector_store.as_retriever()

    if USE_AZURE_OPENAI:
        llm = AzureOpenAI(deployment_name="text-davinci-003")
    else:
        llm = OpenAI(temperature=0)

    chat = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    session_id = str(uuid4())
    chat_history = ElasticsearchChatMessageHistory(
        es_connection=es,
        session_id=session_id,
        index="test_index_chat"
    )


Check: Model is started
Skipping because model is not enabled


In [117]:
# Define a convenience function for Q&A
def ask(question, chat_history):
    result = chat({"question": question, "chat_history": chat_history.messages})
    print(f"""[QUESTION] {question}
[ANSWER]  {result["answer"]}
          [SUPPORTING DOCUMENTS] {list(map(lambda d: d.metadata, list(result["source_documents"])))}""")
    chat_history.add_user_message(result["question"])
    chat_history.add_ai_message(result["answer"])


In [118]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    # Chat away!
    print(f"[CHAT SESSION ID] {session_id}")
    ask("EコマースWebサイトにおいて、Elasticsearchを使ったベストプラクティスはありますか？", chat_history)

Check: Model is started
Skipping because model is not enabled


In [119]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    print(f"[CHAT SESSION ID] {session_id}")
    ask("他にはありますか？", chat_history)

Check: Model is started
Skipping because model is not enabled


In [120]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    print(f"[CHAT SESSION ID] {session_id}")
    ask("他にはありますか？", chat_history)

Check: Model is started
Skipping because model is not enabled


チャットのクリア

In [121]:
if not is_model_started(es, "intfloat__multilingual-e5-base") or os.environ["OPENAI_API_KEY"]  == "":
    print("Skipping because model is not enabled")
else:
    chat_history.clear()

Check: Model is started
Skipping because model is not enabled


# 結果まとめ

In [122]:
for query in questions:
    print(f"query: {query}")
    print("")
    for result in result_list:
        if result['query'] == query:
            print(f"search_logic: {result['search_logic']}")
            pprint(f"{result['result']}")
            print('---------------------------')
    print('==============================')

query: 画像検索の方法について知りたい

search_logic: bm25
"{'query': '画像検索の方法について知りたい', 'result': []}"
---------------------------
search_logic: vector_openai_embeddings
("[{'source': "
 "'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}, "
 "{'source': "
 "'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}, "
 "{'source': "
 "'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}, "
 "{'source': "
 "'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}]")
---------------------------
search_logic: vector_huggingface_e5
("[{'source': "
 "'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}, "
 "{'source': "
 "'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}, "
 "{'source': "
 "'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}, "
 "{'source': "
 "'https://www.elastic.co/jp/blog/how-to-build-scope

# 終了処理

### ELSERモデルの停止

In [134]:
if is_model_started(es, ".elser_model_1"):
    print("Stopping model deployment")
    stop_model(es, es_model_id=".elser_model_1")

Check: Model is started
Stopping model deployment
Stopping model deployment


/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_48614/1723871948.py:77: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  es_connection.options(request_timeout=300).ml.stop_trained_model_deployment(model_id=es_model_id, force=True)


## Elastic ML E5モデルの停止

In [126]:
if is_model_started(es, "intfloat/multilingual-e5-base"):
    stop_model(es, model_id="intfloat/multilingual-e5-base")

Check: Model is started
Stopping model deployment


/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_48614/3028873066.py:77: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  es_connection.options(request_timeout=300).ml.stop_trained_model_deployment(model_id=es_model_id)


## Elastic ML Tohoku BERT モデルの停止

In [127]:
if is_model_started(es, "cl-tohoku/bert-base-japanese-v3"):
    stop_model(es, model_id="cl-tohoku/bert-base-japanese-v3")

Check: Model does not have deployment stats
